# Common Data Preparation 

In [2]:
#pip install pydicom
#!pip install glob
#!pip install pylab
import pandas as pd
import numpy as np
import random
import csv
import os
import pydicom
import glob, pylab
import cv2

from skimage import io
from skimage import measure
from skimage.transform import resize

import tensorflow as tf
from tensorflow import keras

from matplotlib import pyplot as plt
import matplotlib.patches as patches

In [0]:
from google.colab import drive
drive.mount('./content/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./content/


In [0]:
# Code for deleting a non-empty directory 
#import shutil
#shutil.rmtree('data')

In [3]:
project_path = "D:/Datasets/rsna-pneumonia-detection-challenge/"

In [0]:
#from zipfile import ZipFile
#with ZipFile(project_path+'rsna-pneumonia-detection-challenge.zip', 'r') as z:
#  z.extractall()


Load pneumonia class information


In [5]:
# empty dictionary
pneumonia_class = {}
# load table
with open(project_path+'stage_2_train_labels.csv', mode='r') as infile:
    # open reader
    reader = csv.reader(infile)
    # skip header
    next(reader, None)
    # loop through rows
    for rows in reader:
        # retrieve information
        filename = rows[0]
        #location = rows[1:5]
        pneumonia = rows[5]
        
        #add label to dictionary
        if filename not in pneumonia_class:
          pneumonia_class[filename] = {
                                     'label' : pneumonia,
                                      }

In [6]:
len(pneumonia_class)

26684

Load filenames and separate sub-part as validation


In [7]:
# load and shuffle filenames
folder = project_path+'stage_2_train_images'
valid_per = 9
filenames = os.listdir(folder)
random.shuffle(filenames)
# split into train and validation filenames
total_samples = len(filenames)
n_valid_samples = int(total_samples * valid_per / 100)
# n_valid_samples = 2560
train_filenames = filenames[n_valid_samples:]
valid_filenames = filenames[:n_valid_samples]
print('train samples', len(train_filenames))
print('validation samples', len(valid_filenames))
n_train_samples = len(filenames) - n_valid_samples

train samples 24283
validation samples 2401


Create directory structure for train and validation data as per class 

In [8]:
dirNameTrainYes = 'data/train/yes'
os.makedirs(dirNameTrainYes)    

dirNameTrainNo = 'data/train/no'
os.mkdir(dirNameTrainNo)

dirNameValid = 'data/valid'
os.mkdir(dirNameValid)

dirNameValidYes = 'data/valid/yes'
os.mkdir(dirNameValidYes)

dirNameValidNo = 'data/valid/no'
os.mkdir(dirNameValidNo)

dirNameTest = 'data/test'
os.mkdir(dirNameTest)

dirNameTestTest = 'data/test/test'
os.mkdir(dirNameTestTest)

Extract pixel data from DCIM files and place as .png image in appropriate folders - train & validation as per class 

In [10]:
for filename in train_filenames:
  img = pydicom.dcmread(os.path.join(folder, filename)).pixel_array
  filename = filename.split('.')[0]
  # if image contains pneumonia
  if pneumonia_class.get(filename) == {'label': '1'}:
    cv2.imwrite(dirNameTrainYes+'/'+filename+'.png', img)
  else:
    cv2.imwrite(dirNameTrainNo+'/'+filename+'.png', img)


RuntimeError: The following handlers are available to decode the pixel data however they are missing required dependencies: GDCM (req. GDCM), Pillow (req. )

In [0]:
for filename in valid_filenames:
  img = pydicom.dcmread(os.path.join(folder, filename)).pixel_array
  filename = filename.split('.')[0]
  # if image contains pneumonia
  if pneumonia_class.get(filename) == {'label': '1'}:
    cv2.imwrite(dirNameValidYes+'/'+filename+'.png', img)
  else:
    cv2.imwrite(dirNameValidNo+'/'+filename+'.png', img)

In [0]:
# Read Test dicom images and save as png 
TEST_IMG_DIR = 'stage_2_test_images' 
test_files = os.listdir(TEST_IMG_DIR)
test_pIds = [f.split('.')[0] for f in test_files]
#test_files = [os.path.join(TEST_IMG_DIR, f) for f in test_files]


for filename in test_files:
  img = pydicom.dcmread(os.path.join(TEST_IMG_DIR, filename)).pixel_array
  filename = filename.split('.')[0]
  cv2.imwrite(dirNameTestTest+'/'+filename+'.png', img)



In [0]:
print(test_files)

['28209e9c-6bbe-4832-842e-a5b040ce6364.dcm', 'bff10284-7ffa-4e7e-bd19-49a61b36bd94.dcm', '1532c43a-028c-4d77-9cb0-f9d103988e7b.dcm', '21797ffc-8258-415c-b886-5ac6e60e6962.dcm', '0e1d3610-9f6c-43ea-861f-feb86643987a.dcm', '12d70948-c153-45bd-9942-e6cc07d54d07.dcm', '1ebb908b-d5b5-478e-9a91-107476ec3cc2.dcm', '044702b8-5ad7-493c-ac3b-1fe5292a62b2.dcm', '0eec7e74-4d8e-496b-bf47-81f86547ecf4.dcm', '0e74967f-8e25-4916-8133-b527f4df6dcf.dcm', '2b1135f1-c115-45d7-9e92-c2c9779e92b1.dcm', '2a38a2eb-9a5e-479c-9d60-40f55b4f4f36.dcm', '2ea32c58-de2a-4085-bd7c-7b158bd681c8.dcm', '114131f4-3e2a-45cc-b3d1-0876fd9a5ed1.dcm', '2fc5d25d-d8ba-4c44-be7d-a665911d2191.dcm', '11c194f2-c78e-4a1d-aed4-e1fdb7cfba3f.dcm', '2b989809-abd6-4430-9c8f-4805723d851b.dcm', '14b8f48d-f637-47dc-8d93-5ffb65ca2e18.dcm', 'c038896c-a4d7-4e16-9cc7-d6a5d89f8012.dcm', '0028450f-5b8e-4695-9416-8340b6f686b0.dcm', '046a9f1a-abb6-4e75-94f5-a98a197a3ee9.dcm', '24c49b21-8b28-482e-9d8f-676a93811b83.dcm', '255f3ba9-89bc-4ce4-840d-432676

# Detection Model Training

Data Preparation

In [0]:
#train_filenames = filenames[n_valid_samples:]
#valid_filenames = filenames[:n_valid_samples]

In [0]:
train_filenames_df = pd.DataFrame(data=train_filenames, columns=['patientId'])
valid_filenames_df = pd.DataFrame(data=valid_filenames, columns=['patientId'])

In [0]:
train_filenames_df.head()

,patientId
0,b572006e-3f9e-4608-b27c-1e492ae63734.dcm
1,de04a499-1e5c-4a5a-becd-4c6069a7e518.dcm
2,bf3ef845-34ab-4b80-81e7-ac7a3b23485a.dcm
3,1328013e-dca0-497b-9523-072bc6b23e17.dcm
4,f938d936-8c4a-497e-9b73-244a22786490.dcm


In [0]:
len(train_filenames_df)

24283

In [0]:
valid_filenames_df.head()

,patientId
0,5674f428-78fa-4e0c-8ecb-b5ecc2d83018.dcm
1,3827ecef-1d37-421e-ad89-2738010bccd1.dcm
2,0695a0d8-d4b6-4dbe-bc04-497897e4516a.dcm
3,ba614c4f-bb2a-4153-9a46-1927aa826585.dcm
4,8cc0b6f0-ca96-47a8-bf47-7fe2d3641d0c.dcm


In [0]:
len(valid_filenames_df)

2401

In [0]:
Pneumonia_df = pd.read_csv('stage_2_train_labels.csv')

In [0]:
Pneumonia_df.head()

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


Merge to form Train dataset

In [0]:
train_filenames_df['patientId'] = train_filenames_df['patientId'].str.replace(r'.dcm$', '')
train_filenames_df.head()

,patientId
0,b572006e-3f9e-4608-b27c-1e492ae63734
1,de04a499-1e5c-4a5a-becd-4c6069a7e518
2,bf3ef845-34ab-4b80-81e7-ac7a3b23485a
3,1328013e-dca0-497b-9523-072bc6b23e17
4,f938d936-8c4a-497e-9b73-244a22786490


In [0]:
# Merge datasets using inner join on common field "patientId" 

Pneumonia_train_df = pd.merge(left=Pneumonia_df, right=train_filenames_df, how='inner', on='patientId')
Pneumonia_train_df = Pneumonia_train_df[Pneumonia_train_df['Target']==1]
Pneumonia_train_df['patientId'] = dirNameTrainYes+'/'+Pneumonia_train_df['patientId']+'.png'
Pneumonia_train_df.head()

,patientId,x,y,width,height,Target
5,data/train/yes/00704310-78a8-4b38-8475-49f4573...,323.0,577.0,160.0,104.0,1
6,data/train/yes/00704310-78a8-4b38-8475-49f4573...,695.0,575.0,162.0,137.0,1
11,data/train/yes/00aecb01-a116-45a2-956c-08d2fa5...,288.0,322.0,94.0,135.0,1
12,data/train/yes/00aecb01-a116-45a2-956c-08d2fa5...,547.0,299.0,119.0,165.0,1
13,data/train/yes/00c0b293-48e7-4e16-ac76-9269ba5...,306.0,544.0,168.0,244.0,1


In [0]:
len(Pneumonia_train_df)

8718

Merge to form Validation dataset

In [0]:
valid_filenames_df['patientId'] = valid_filenames_df['patientId'].str.replace(r'.dcm$', '')
valid_filenames_df.head()

,patientId
0,5674f428-78fa-4e0c-8ecb-b5ecc2d83018
1,3827ecef-1d37-421e-ad89-2738010bccd1
2,0695a0d8-d4b6-4dbe-bc04-497897e4516a
3,ba614c4f-bb2a-4153-9a46-1927aa826585
4,8cc0b6f0-ca96-47a8-bf47-7fe2d3641d0c


In [0]:
# Merge datasets using inner join on common field "patientId" 

Pneumonia_valid_df = pd.merge(left=Pneumonia_df, right=valid_filenames_df, how='inner', on='patientId')
Pneumonia_valid_df = Pneumonia_valid_df[Pneumonia_valid_df['Target']==1]
Pneumonia_valid_df['patientId'] = dirNameValidYes+'/'+Pneumonia_valid_df['patientId']+'.png'
Pneumonia_valid_df.head()

,patientId,x,y,width,height,Target
0,data/valid/yes/00436515-870c-4b36-a041-de91049...,264.0,152.0,213.0,379.0,1
1,data/valid/yes/00436515-870c-4b36-a041-de91049...,562.0,152.0,256.0,453.0,1
3,data/valid/yes/01b9e362-4950-40f5-88fa-7557ac2...,366.0,289.0,208.0,527.0,1
4,data/valid/yes/01b9e362-4950-40f5-88fa-7557ac2...,714.0,278.0,255.0,576.0,1
5,data/valid/yes/01be392f-a46d-4aef-a57e-9cd1a80...,535.0,626.0,177.0,240.0,1


In [0]:
len(Pneumonia_valid_df)

837

Write Training CSV

In [0]:
Pneumonia_train_df.to_csv(r'train.csv', index = None, header=False) 


Write Validation CSV

In [0]:
Pneumonia_valid_df.to_csv(r'validation.csv', index = None, header=False) 

Initiation for Detection Model

Reference for model, generator and loss functions
 
https://github.com/lars76/object-localization/blob/master/example_4/train.py

In [0]:
import numpy as np
import tensorflow as tf
import math 
from PIL import Image, ImageDraw, ImageEnhance
from tensorflow.keras import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, BatchNormalization, Activation
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.backend import epsilon
from tensorflow.keras.models import model_from_json

# 0.35, 0.5, 0.75, 1.0
ALPHA = 0.35

GRID_SIZE = 2           # Changed from original 7 to 2 
IMAGE_SIZE = 512

# first train with frozen weights, then fine tune
TRAINABLE = False
WEIGHTS = "model-0.64.h5"

EPOCHS = 20
BATCH_SIZE = 32
PATIENCE = 15
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 0.0005
LR_DECAY = 0.0001

MULTITHREADING = False
THREADS = 1

TRAIN_CSV = "train.csv"
VALIDATION_CSV = "validation.csv"


Custom Data Generator

In [0]:
class DataGenerator(Sequence):

    def __init__(self, csv_file, rnd_rescale=True, rnd_multiply=True, rnd_color=True, rnd_crop=True, rnd_flip=False, debug=False):
        self.boxes = []
        self.rnd_rescale = rnd_rescale
        self.rnd_multiply = rnd_multiply
        self.rnd_color = rnd_color
        self.rnd_crop = rnd_crop
        self.rnd_flip = rnd_flip
        self.debug = debug

        with open(csv_file, "r") as file:
            reader = csv.reader(file, delimiter=",")
            for index, row in enumerate(reader):
                for i, r in enumerate(row[1:5]):
                    row[i+1] = int(float(r)) 

                image_height=1024 
                image_width=1024 
                path, x0, y0, x1, y1, _ = row
                self.boxes.append((path, x0, y0, x1, y1))

    def __len__(self):
        return math.ceil(len(self.boxes) / BATCH_SIZE)

    def __getitem__(self, idx):
        boxes = self.boxes[idx * BATCH_SIZE:(idx + 1) * BATCH_SIZE]

        batch_images = np.zeros((len(boxes), IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
        batch_boxes = np.zeros((len(boxes), GRID_SIZE, GRID_SIZE, 5), dtype=np.float32)
        for i, row in enumerate(boxes):
            path, x0, y0, x1, y1 = row

            with Image.open(path) as img:
                if self.rnd_rescale:
                    old_width = img.width
                    old_height = img.height

                    rescale = np.random.uniform(low=0.6, high=1.4)
                    new_width = int(old_width * rescale)
                    new_height = int(old_height * rescale)

                    img = img.resize((new_width, new_height))

                    x0 *= new_width / old_width
                    y0 *= new_height / old_height
                    x1 *= new_width / old_width
                    y1 *= new_height / old_height

                if self.rnd_crop:
                    start_x = np.random.randint(0, high=np.floor(0.15 * img.width))
                    stop_x = img.width - np.random.randint(0, high=np.floor(0.15 * img.width))
                    start_y = np.random.randint(0, high=np.floor(0.15 * img.height))
                    stop_y = img.height - np.random.randint(0, high=np.floor(0.15 * img.height))

                    img = img.crop((start_x, start_y, stop_x, stop_y))

                    x0 = max(x0 - start_x, 0)
                    y0 = max(y0 - start_y, 0)
                    x1 = min(x1 - start_x, img.width)
                    y1 = min(y1 - start_y, img.height)

                    #if np.abs(x1 - x0) < 5 or np.abs(y1 - y0) < 5:
                    #    print("\nWarning: cropped too much (obj width {}, obj height {}, img width {}, img height {})\n".format(x1 - x0, y1 - y0, img.width, img.height))

                if self.rnd_flip:
                    elem = np.random.choice([0, 90, 180, 270, 1423, 1234])
                    if elem % 10 == 0:
                        x = x0 - img.width / 2
                        y = y0 - img.height / 2

                        x0 = img.width / 2 + x * np.cos(np.deg2rad(elem)) - y * np.sin(np.deg2rad(elem))
                        y0 = img.height / 2 + x * np.sin(np.deg2rad(elem)) + y * np.cos(np.deg2rad(elem))

                        x = x1 - img.width / 2
                        y = y1 - img.height / 2

                        x1 = img.width / 2 + x * np.cos(np.deg2rad(elem)) - y * np.sin(np.deg2rad(elem))
                        y1 = img.height / 2 + x * np.sin(np.deg2rad(elem)) + y * np.cos(np.deg2rad(elem))

                        img = img.rotate(-elem)
                    else:
                        if elem == 1423:
                            img = img.transpose(Image.FLIP_TOP_BOTTOM)
                            y0 = img.height - y0
                            y1 = img.height - y1

                        elif elem == 1234:
                            img = img.transpose(Image.FLIP_LEFT_RIGHT)
                            x0 = img.width - x0
                            x1 = img.width - x1

                image_width = img.width
                image_height = img.height

                tmp = x0
                x0 = min(x0, x1)
                x1 = max(tmp, x1)

                tmp = y0
                y0 = min(y0, y1)
                y1 = max(tmp, y1)

                x0 = max(x0, 0)
                y0 = max(y0, 0)

                y0 = min(y0, image_height)
                x0 = min(x0, image_width)
                y1 = min(y1, image_height)
                x1 = min(x1, image_width)

                if self.rnd_color:
                    enhancer = ImageEnhance.Color(img)
                    img = enhancer.enhance(np.random.uniform(low=0.5, high=1.5))

                    enhancer2 = ImageEnhance.Brightness(img)
                    img = enhancer.enhance(np.random.uniform(low=0.7, high=1.3))

                img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
                img = img.convert('RGB')
                img = np.array(img, dtype=np.float32)

                if self.rnd_multiply:
                    img[...,0] = np.floor(np.clip(img[...,0] * np.random.uniform(low=0.8, high=1.2), 0.0, 255.0))
                    img[...,1] = np.floor(np.clip(img[...,1] * np.random.uniform(low=0.8, high=1.2), 0.0, 255.0))
                    img[...,2] = np.floor(np.clip(img[...,2] * np.random.uniform(low=0.8, high=1.2), 0.0, 255.0))

                batch_images[i] = preprocess_input(img.copy())

            x_c = (GRID_SIZE / image_width) * (x0 + (x1 - x0) / 2)
            y_c = (GRID_SIZE / image_height) * (y0 + (y1 - y0) / 2)

            floor_y = math.floor(y_c)
            floor_x = math.floor(x_c)

            batch_boxes[i, floor_y, floor_x, 0] = (y1 - y0) / image_height
            batch_boxes[i, floor_y, floor_x, 1] = (x1 - x0) / image_width
            batch_boxes[i, floor_y, floor_x, 2] = y_c - floor_y
            batch_boxes[i, floor_y, floor_x, 3] = x_c - floor_x
            batch_boxes[i, floor_y, floor_x, 4] = 1

            if self.debug:
                changed = img.astype(np.uint8)
                if not os.path.exists("__debug__"):
                    os.makedirs("__debug__")

                changed = Image.fromarray(changed)

                x_c = (floor_x + batch_boxes[i, floor_y, floor_x, 3]) / GRID_SIZE
                y_c = (floor_y + batch_boxes[i, floor_y, floor_x, 2]) / GRID_SIZE

                y0 = IMAGE_SIZE * (y_c - batch_boxes[i, floor_y, floor_x, 0] / 2)
                x0 = IMAGE_SIZE * (x_c - batch_boxes[i, floor_y, floor_x, 1] / 2)
                y1 = y0 + IMAGE_SIZE * batch_boxes[i, floor_y, floor_x, 0]
                x1 = x0 + IMAGE_SIZE * batch_boxes[i, floor_y, floor_x, 1]

                draw = ImageDraw.Draw(changed)
                draw.rectangle(((x0, y0), (x1, y1)), outline="green")

                changed.save(os.path.join("__debug__", os.path.basename(path)))

        return batch_images, batch_boxes

Validation Function

In [0]:
class Validation(Callback):
    def get_box_highest_percentage(self, mask):
        reshaped = mask.reshape(mask.shape[0], np.prod(mask.shape[1:-1]), -1)

        score_ind = np.argmax(reshaped[...,-1], axis=-1)
        unraveled = np.array(np.unravel_index(score_ind, mask.shape[:-1])).T[:,1:]

        cell_y, cell_x = unraveled[...,0], unraveled[...,1]
        boxes = mask[np.arange(mask.shape[0]), cell_y, cell_x]

        h, w, offset_y, offset_x = boxes[...,0], boxes[...,1], boxes[...,2], boxes[...,3]

        return np.stack([cell_y + offset_y, cell_x + offset_x,
                        GRID_SIZE * h, GRID_SIZE * w], axis=-1)

    def __init__(self, generator):
        self.generator = generator

    def on_epoch_end(self, epoch, logs):
        mse = 0
        intersections = 0
        unions = 0

        for i in range(len(self.generator)):
            batch_images, gt = self.generator[i]
            pred = self.model.predict_on_batch(batch_images)  # in tf2.0: add .numpy()

            pred = self.get_box_highest_percentage(pred)
            gt = self.get_box_highest_percentage(gt)

            mse += np.linalg.norm(gt - pred, ord='fro') / pred.shape[0]

            pred = np.maximum(pred, 0)
            gt = np.maximum(gt, 0)

            diff_height = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
            diff_width = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
            intersection = np.maximum(diff_width, 0) * np.maximum(diff_height, 0)

            area_gt = gt[:,2] * gt[:,3]
            area_pred = pred[:,2] * pred[:,3]
            union = np.maximum(area_gt + area_pred - intersection, 0)

            intersections += np.sum(intersection * (union > 0))
            unions += np.sum(union)

        iou = np.round(intersections / (unions + epsilon()), 4)
        logs["val_iou"] = iou

        mse = np.round(mse, 4)
        logs["val_mse"] = mse

        print(" - val_iou: {} - val_mse: {}".format(iou, mse))


Loss Function

In [0]:
def detection_loss():
    def get_box_highest_percentage(arr):
        shape = tf.shape(arr)

        reshaped = tf.reshape(arr, (shape[0], tf.reduce_prod(shape[1:-1]), -1))

        # returns array containing the index of the highest percentage of each batch
        # where 0 <= index <= height * width
        max_prob_ind = tf.argmax(reshaped[...,-1], axis=-1, output_type=tf.int32)

        # turn indices (batch, y * x) into (batch, y, x)
        # returns (3, batch) tensor
        unraveled = tf.unravel_index(max_prob_ind, shape[:-1])

        # turn tensor into (batch, 3) and keep only (y, x)
        unraveled = tf.transpose(unraveled)[:,1:]
        y, x = unraveled[...,0], unraveled[...,1]

        # stack indices and create (batch, 5) tensor which
        # contains height, width, offset_y, offset_x, percentage
        indices = tf.stack([tf.range(shape[0]), y, x], axis=-1)
        box = tf.gather_nd(arr, indices)

        y, x = tf.cast(y, tf.float32), tf.cast(x, tf.float32)

        # transform box to (y + offset_y, x + offset_x, 7 * height, 7 * width, obj)
        # output is (batch, 5)
        out = tf.stack([y + box[...,2], x + box[...,3],
                        GRID_SIZE * box[...,0], GRID_SIZE * box[...,1],
                        box[...,-1]], axis=-1)

        return out

    def loss(y_true, y_pred):
        # get the box with the highest percentage in each image
        true_box = get_box_highest_percentage(y_true)
        pred_box = get_box_highest_percentage(y_pred)

        # object loss
        obj_loss = binary_crossentropy(y_true[...,4:5], y_pred[...,4:5])

        # mse with the boxes that have the highest percentage
        box_loss = tf.reduce_sum(tf.math.squared_difference(true_box[...,:-1], pred_box[...,:-1]))

        return tf.reduce_sum(obj_loss) + box_loss

    return loss

Create Model

In [0]:
def create_model(trainable=False):
    model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, alpha=ALPHA, weights="imagenet")

    for layer in model.layers:
        layer.trainable = trainable

    block = model.get_layer("block_16_project_BN").output

    x = Conv2D(112, (3,3), padding="same",  strides=4, activation="relu")(block)
    x = Conv2D(112, (3,3), padding="same",  strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(5, padding="same", kernel_size=1, activation="sigmoid")(x)

    model = Model(inputs=model.input, outputs=x)

    # divide by 2 since d/dweight learning_rate * weight^2 = 2 * learning_rate * weight
    # see https://arxiv.org/pdf/1711.05101.pdf
    regularizer = l2(WEIGHT_DECAY / 2)
    for weight in model.trainable_weights:
        with tf.keras.backend.name_scope("weight_regularizer"):
            model.add_loss(regularizer(weight)) # in tf2.0: lambda: regularizer(weight)

    return model

Compile Model

In [0]:
model = create_model(trainable=TRAINABLE)
model.summary()

if TRAINABLE:
  model.load_weights(WEIGHTS)

train_datagen = DataGenerator(TRAIN_CSV)

val_generator = DataGenerator(VALIDATION_CSV, rnd_rescale=False, rnd_multiply=False, rnd_crop=False, rnd_flip=False, debug=False)
validation_datagen = Validation(generator=val_generator)

learning_rate = LEARNING_RATE
if TRAINABLE:
  learning_rate /= 10

optimizer = SGD(lr=learning_rate, decay=LR_DECAY, momentum=0.9, nesterov=False)
model.compile(loss=detection_loss(), optimizer=optimizer, metrics=[])

checkpoint = ModelCheckpoint("model-{val_iou:.2f}.h5", monitor="val_iou", verbose=1, save_best_only=True,
                                 save_weights_only=True, mode="max")
stop = EarlyStopping(monitor="val_iou", patience=PATIENCE, mode="max")
reduce_lr = ReduceLROnPlateau(monitor="val_iou", factor=0.6, patience=5, min_lr=1e-6, verbose=1, mode="max")


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 513, 513, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 256, 256, 16) 432         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 256, 256, 16) 64          Conv1[0][0]                      
____________________________________________________________________________________________

Fit Model

In [0]:
model.fit_generator(generator=train_datagen,
                        epochs=EPOCHS,
                        callbacks=[validation_datagen, checkpoint, reduce_lr, stop],
                        workers=THREADS,
                        use_multiprocessing=MULTITHREADING,
                        shuffle=True,
                        verbose=1)

Epoch 1/20
272/273 [============================>.] - ETA: 0s - loss: 19.0140 - val_iou: 0.1154 - val_mse: 2.8585

Epoch 00001: val_iou improved from -inf to 0.11540, saving model to model-0.12.h5
273/273 [==============================] - 361s 1s/step - loss: 18.9806
Epoch 2/20
272/273 [============================>.] - ETA: 0s - loss: 15.2182 - val_iou: 0.1183 - val_mse: 2.9951

Epoch 00002: val_iou improved from 0.11540 to 0.11830, saving model to model-0.12.h5
273/273 [==============================] - 251s 919ms/step - loss: 15.2390
Epoch 3/20
272/273 [============================>.] - ETA: 0s - loss: 14.4230 - val_iou: 0.1358 - val_mse: 2.9509

Epoch 00003: val_iou improved from 0.11830 to 0.13580, saving model to model-0.14.h5
273/273 [==============================] - 249s 911ms/step - loss: 14.4186
Epoch 4/20
272/273 [============================>.] - ETA: 0s - loss: 14.5276 - val_iou: 0.1051 - val_mse: 2.8775

Epoch 00004: val_iou did not improve from 0.13580
273/273 [=======

KeyboardInterrupt: ignored

# Detection Model Prediction

In [0]:
import cv2            
import glob
import numpy as np
# from google.colab.patches import cv2_imshow

WEIGHTS_FILE = "model-0.14.h5" # To be updated 
IMAGES = "data/test/test/*png"

IOU_THRESHOLD = 0.45
SCORE_THRESHOLD = 0.1
MAX_OUTPUT_SIZE = 4



In [0]:
submission_df  = pd.DataFrame(columns = ['patientId', 'PredictionString'])

In [0]:

# Dependencies 
# Test tree structure 
# Detection model definition 

# Uncomment these two lines if model to be created and weights loaded  
model = create_model()
model.load_weights(WEIGHTS_FILE)

test_cnt = 0
for filename in glob.glob(IMAGES):

  test_cnt += 1
  print("Processing Test Image ",test_cnt)

  True_filename = filename.replace(r'data/test/test/', '')
  #class_pred['PatientImage'] = class_pred['PatientImage'].str.replace(r'test/', '')
  #result_df = class_pred.loc[class_pred['PatientImage'] == True_filename] 
  #PID = result_df.iloc[0]['PatientImage']
  #PID = PID.replace(r'.png$', '')
  True_filename = True_filename.replace(r'.png', '')

  # If no Pneumonia predicted in classification, then do not predict
  # Assuming 'Class1' means Pneumonia 
  # if(result_df.iloc[0]['Class0'] > result_df.iloc[0]['Class1']):
  #   submission_df = submission_df.append({'patientId' : PID , 'PredictionString' : ''} , ignore_index=True)
  #   continue

  unscaled = cv2.imread(filename)
  img = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))

  feat_scaled = preprocess_input(np.array(img, dtype=np.float32))

  pred = np.squeeze(model.predict(feat_scaled[np.newaxis,:]))
  height, width, y_f, x_f, score = [a.flatten() for a in np.split(pred, pred.shape[-1], axis=-1)]

  coords = np.arange(pred.shape[0] * pred.shape[1])
  y = (y_f + coords // pred.shape[0]) / pred.shape[0]
  x = (x_f + coords % pred.shape[1]) / pred.shape[1]

  boxes = np.stack([y, x, height, width, score], axis=-1)
  boxes = boxes[np.where(boxes[...,-1] >= SCORE_THRESHOLD)]

  selected_indices = tf.image.non_max_suppression(boxes[...,:-1], boxes[...,-1], MAX_OUTPUT_SIZE, IOU_THRESHOLD)
  selected_indices = tf.Session().run(selected_indices)

  PredStr = ''

  for y_c, x_c, h, w, scr in boxes[selected_indices]:
    x0 = unscaled.shape[1] * (x_c - w / 2)
    y0 = unscaled.shape[0] * (y_c - h / 2)
    w = unscaled.shape[1] * w
    h = unscaled.shape[0] * h
    PredStr +=  str(scr) + " " + str(x0) + " " + str(y0) + " " + str(w) + " " + str(h) + " "

  submission_df = submission_df.append({'patientId' : True_filename, 'PredictionString' : PredStr} , ignore_index=True)
    #cv2.rectangle(unscaled, (int(x0), int(y0)), (int(x1), int(y1)), (0, 255, 0), 1)



Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


2023424/2019640 [==============================] - 2s 1us/step
Processing Test Image  1
Processing Test Image  2
Processing Test Image  3
Processing Test Image  4
Processing Test Image  5
Processing Test Image  6
Processing Test Image  7
Processing Test Image  8
Processing Test Image  9
Processing Test Image  10
Processing Test Image  11
Processing Test Image  12
Processing Test Image  13
Processing Test Image  14
Processing Test Image  15
Processing Test Image  16
Processing Test Image  17
Processing Test Image  18
Processing Test Image  19
Processing Test Image  20
Processing Test Image  21
Processing Test Image  22
Processing Test Image  23
Processing Test Image  24
Processing Test Image  25
Processing Test Image  26
Processing Test Image  27
Processing Test Image  28
Processing Test Image  29
Processing Test Image  30
Processing Test Image  31
Processing Test Image  32
Processing Test Image  33
Processing Test Image  34
Processing Test Image  35
Processing Test Image  36
Processing

In [0]:
submission_df.head(50)       

,patientId,PredictionString
0,23fef1ae-6472-4def-948b-5594056f2b79,0.9861680865287781 140.62933349609375 117.0159...
1,28cea9d9-0b70-4a9c-945d-d321bf809c8d,0.9851769804954529 146.989990234375 125.976791...
2,2a5e9649-6635-4f85-96da-1f25e96b2528,0.984532356262207 128.7378387451172 124.484039...
3,2e1f64dc-4b8f-4cfb-950d-800abdbc833e,0.9437884092330933 106.16940307617188 100.7209...
4,2e600572-801d-40be-a6aa-627baad344dd,0.9842451810836792 142.22311401367188 122.5114...
5,143e47f2-3d11-44a6-a9eb-b0aaf291db55,0.9879639148712158 156.04391479492188 128.3855...
6,0146516d-b739-4103-8cc6-bb2b494e1d5b,0.9912663698196411 180.15841674804688 93.70597...
7,1d023143-2c5b-448f-94e2-c140f47b10ab,0.9829754829406738 130.82574462890625 115.4826...
8,1328d56a-5b37-41b2-bc1b-a6c90989c642,0.9125599265098572 90.876953125 68.60668945312...
9,297be3aa-dcb5-4e28-9aed-69ac55d3dd41,0.9855694770812988 151.8672637939453 156.06077...


In [0]:
submission_df.to_csv(r'Submission.csv', index = None, header=True) 